In [0]:
%sql
USE CATALOG field_demos;
USE ml_ops;

-- Create or replace the online_retail_curated table with cleaned and consistent descriptions
CREATE OR REPLACE TABLE online_retail_curated AS

WITH date_shift AS (
    -- Get the max InvoiceDate and today's date
    SELECT 
        MAX(CAST(InvoiceDate AS DATE)) AS max_date,
        CURRENT_DATE() AS today_date
    FROM online_retail
),
shifted_data AS (
    -- Shift InvoiceDate so that the max date equals today's date
    SELECT 
        *,
        DATE_ADD(CAST(InvoiceDate AS DATE), 
                 DATEDIFF(CURRENT_DATE(), (SELECT max_date FROM date_shift))) AS shifted_InvoiceDate
    FROM online_retail
),
chosen_description AS (
    -- For each StockCode, choose a valid description if available; otherwise, use 'NO_DESCRIPTION'
    SELECT 
        TRIM(UPPER(CAST(StockCode AS STRING))) AS StockCode,
        CASE 
          WHEN COUNT(CASE 
                        WHEN Description IS NOT NULL 
                             AND UPPER(TRIM(Description)) NOT IN ('', 'NAN', 'NULL') 
                        THEN 1 END) > 0
          THEN MIN(CASE 
                      WHEN Description IS NOT NULL 
                           AND UPPER(TRIM(Description)) NOT IN ('', 'NAN', 'NULL') 
                      THEN TRIM(UPPER(Description)) END)
          ELSE 'NO_DESCRIPTION'
        END AS ChosenDescription
    FROM shifted_data
    WHERE StockCode IS NOT NULL AND StockCode != ''
    GROUP BY TRIM(UPPER(CAST(StockCode AS STRING)))
)
SELECT DISTINCT
    TRIM(UPPER(CAST(sd.StockCode AS STRING))) AS StockCode,
    cd.ChosenDescription AS Description,
    CASE 
        WHEN sd.Quantity IS NULL OR sd.Quantity < 0 THEN 0 
        ELSE sd.Quantity 
    END AS Quantity,
    sd.shifted_InvoiceDate AS InvoiceDate,
    sd.UnitPrice,
    sd.CustomerID,
    sd.Country
FROM shifted_data sd
LEFT JOIN chosen_description cd
    ON TRIM(UPPER(sd.StockCode)) = cd.StockCode
WHERE sd.StockCode IS NOT NULL AND sd.StockCode != '';

-- After creating the table, show the distinct counts for verification
SELECT 
    COUNT(DISTINCT CONCAT(StockCode, '_', Description)) AS Distinct_StockCode_Description,
    COUNT(DISTINCT StockCode) AS Distinct_StockCode,
    COUNT(DISTINCT Description) AS Distinct_Description
FROM online_retail_curated;